<a href="https://colab.research.google.com/github/SSHDMU0001/WooriFis-AIU/blob/main/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

rating_data = pd.read_csv('./movie_lens/ratings.csv')
movie_data = pd.read_csv('./movie_lens/movies.csv')

rating_data.head(2)

movie_data.head(2)

rating_data.drop('timestamp', axis = 1, inplace=True)
rating_data.head(2)

user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')

user_movie_rating.head(2)

movie_user_rating = user_movie_rating.pivot_table('rating', index = 'title', columns='userId')
user_movie_rating = user_movie_rating.pivot_table('rating', index = 'userId', columns='title')

user_movie_rating.head(5)

movie_user_rating.head()

movie_user_rating.fillna(0, inplace = True)
movie_user_rating.head(3)

item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor

print(movie_user_rating.shape)
print(item_based_collabor.shape)

item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)

item_based_collabor.head()

def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]


get_item_based_collabor('Godfather, The (1972)')

(9064, 671)
(9064, 9064)


title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.773685
Goodfellas (1990)                            0.620349
One Flew Over the Cuckoo's Nest (1975)       0.568244
American Beauty (1999)                       0.557997
Star Wars: Episode IV - A New Hope (1977)    0.546750
Name: Godfather, The (1972), dtype: float64